In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import torch
print ('number of GPUs:', torch.cuda.device_count())

from tqdm import tqdm
from matplotlib import pyplot as plt 

from minichatgpt.experiments.imdb import config, sent_kwargs
from minichatgpt import Lab
from minichatgpt.processdata.collators import imdb_dataloader_collator

# For the sake of the speed of this demonstration, the batch_size is temporarily decreased from 256 to 4
batch_size = 128
config.batch_size = batch_size
config.forward_batch_size = batch_size//2
config.seed

/usr/bin/nvidia-modprobe: unrecognized option: "-s"

ERROR: Invalid commandline, please run `/usr/bin/nvidia-modprobe --help` for usage information.


/usr/bin/nvidia-modprobe: unrecognized option: "-s"

ERROR: Invalid commandline, please run `/usr/bin/nvidia-modprobe --help` for usage information.




number of GPUs: 2


0

In [ ]:
lab = Lab(config)

dataset = lab.build_dataset(dataset_name="imdb",input_min_text_length=2,input_max_text_length=8)

new_policy, old_policy, tokenizer = lab.init_policies_tokenizer()

lab.set_generation_config(do_sample=True,output_min_length=4,output_max_length=16,pad_token_id=tokenizer.eos_token_id)

ppo_trainer = lab.init_ppo_trainer(
    config, 
    new_policy, 
    old_policy, 
    tokenizer, 
    dataset, 
    dataloader_collator = imdb_dataloader_collator,
)

reward_model = lab.init_reward_model()

In [ ]:
returns_list = []
rewards_list = []
print(new_policy.pretrained_model.device)
print(new_policy.v_head)
new_policy.v_head.summary.to('cuda:1')
print(new_policy.v_head.summary.weight.device)

In [ ]:
new_policy.pretrained_model.parallelize()

In [ ]:
lab.reward_model.device

In [ ]:
for batch_step, batch in tqdm(enumerate(ppo_trainer.dataloader)):
    
    queries = batch['input_ids']
    
    #### Get response from gpt2
    responses = []
    for query in queries:
        gen_len = lab.output_length_sampler()
        lab.generation_kwargs["max_new_tokens"] = gen_len
        response = ppo_trainer.generate(query, **lab.generation_kwargs)
        responses.append(response.squeeze()[-gen_len:])
        
    batch['response'] = [tokenizer.decode(r.squeeze()) for r in responses]
    
    #### Compute sentiment score
    texts = [q + r for q,r in zip(batch['query'], batch['response'])]
    pipe_outputs = lab.reward_model(texts, **sent_kwargs)
    rewards = [torch.tensor(output[1]["score"]) for output in pipe_outputs]
    rewards_list.append(torch.mean(rewards).item())
    rewards = [t.to(new_policy.pretrained_model.device) for t in rewards]
    
    #### Run PPO step 
    stats = ppo_trainer.step(queries, responses, rewards)
    ppo_trainer.log_stats(stats, batch, rewards)
    
    #break
    
    returns_list.append(stats['ppo/returns/mean'])

In [ ]:
plt.figure()
plt.xlabel('Training Iterations (minibatches)', fontsize=25)
plt.ylabel('Mean Sum of Rewards (per Sequence)', fontsize=25)
plt.plot(returns_list, label='returns')
plt.plot(rewards_list, label='rewards')
plt.legend()
plt.show()